### Qinhui Xu 09/19/2018

#### Data Source: https://www.kaggle.com/mlg-ulb/creditcardfraud

As I mentioned in dealing with imbalance step shows, I will prepare two dataset - one come from cluster based undersampling - the other one come from only SMOTE method.

I will use following different models to make predictions:

1. Logistic Regression
2. Gradient Boosting Machine
3. Random Forest Model
4. XGBoost
5. Neural Network

#### Data Preprocessing

- Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score,accuracy_score
from sklearn.metrics import confusion_matrix

C:\Users\Tiffany Xu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Data Preprocessing based on Data Exploration / Feature Engineering
df = pd.read_csv("C:/Users/Tiffany Xu/Documents/MachineLearningStudy/DeepLearning/creditcard.csv")

df['V1_'] = df.V1.map(lambda x: 1 if x < -3 else 0)
df['V2_'] = df.V2.map(lambda x: 1 if x > 2.5 else 0)
df['V3_'] = df.V3.map(lambda x: 1 if x < -4 else 0)
df['V4_'] = df.V4.map(lambda x: 1 if x > 2.5 else 0)
df['V5_'] = df.V5.map(lambda x: 1 if x < -4.5 else 0)
df['V6_'] = df.V6.map(lambda x: 1 if x < -2.5 else 0)
df['V7_'] = df.V7.map(lambda x: 1 if x < -3 else 0)
df['V9_'] = df.V9.map(lambda x: 1 if x < -2 else 0)
df['V10_'] = df.V10.map(lambda x: 1 if x < -2.5 else 0)
df['V11_'] = df.V11.map(lambda x: 1 if x > 2 else 0)
df['V12_'] = df.V12.map(lambda x: 1 if x < -2 else 0)
df['V14_'] = df.V14.map(lambda x: 1 if x < -2.5 else 0)
df['V16_'] = df.V16.map(lambda x: 1 if x < -2 else 0)
df['V17_'] = df.V17.map(lambda x: 1 if x < -2 else 0)
df['V18_'] = df.V18.map(lambda x: 1 if x < -2 else 0)
df['V19_'] = df.V19.map(lambda x: 1 if x > 1.5 else 0)
df['V21_'] = df.V21.map(lambda x: 1 if x > 0.6 else 0)

df['normalized_amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
df = df.drop(['Amount','Time'], axis=1)

df_nn = df
df = df.drop(['V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8'], axis =1)

- Cluster Based on Undersampling

In [3]:
##### K-mean Cluster Based on Undersampling
import h2o
h2o.init()
import imp
from h2o.estimators.kmeans import H2OKMeansEstimator

df_notfraud = df[df["Class"]==0]
df_fraud = df[df["Class"]==1]

df_kmeans_notfraud = df_notfraud.drop(['Class'], axis=1)
hf = h2o.H2OFrame(df_kmeans_notfraud)
cls = H2OKMeansEstimator(k=len(df_fraud), standardize=True)
cls.train(x=hf.columns, training_frame=hf)

df_centers = pd.DataFrame(cls.centers())
df_centers.columns = hf.columns
df_centers["Class"] = 0
df_kmeans_undersample = pd.concat([df_centers,df_fraud],axis=0)

X_Kmeans_under = df_kmeans_undersample.loc[:,df_kmeans_undersample.columns != 'Class']
y_Kmeans_under = df_kmeans_undersample.loc[:,df_kmeans_undersample.columns == 'Class']

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from C:\Users\Tiffany Xu\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\TIFFAN~1\AppData\Local\Temp\tmpc22jfn45
  JVM stdout: C:\Users\TIFFAN~1\AppData\Local\Temp\tmpc22jfn45\h2o_Tiffany_Xu_started_from_python.out
  JVM stderr: C:\Users\TIFFAN~1\AppData\Local\Temp\tmpc22jfn45\h2o_Tiffany_Xu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,06 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.2
H2O cluster version age:,3 months and 4 days
H2O cluster name:,H2O_from_python_Tiffany_Xu_mvrztd
H2O cluster total nodes:,1
H2O cluster free memory:,3.492 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%


In [4]:
df_X_Kmenas = pd.DataFrame(X_Kmeans_under,columns = df_kmeans_notfraud.columns)
df_Y_Kmenas = pd.DataFrame(y_Kmeans_under,columns = ["Class"])
df_Kmeans = pd.concat([df_X_Kmenas,df_Y_Kmenas],axis=1)

In [5]:
df_Kmeans.head()

,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,...,V11_,V12_,V14_,V16_,V17_,V18_,V19_,V21_,normalized_amount,Class
0,-0.727143,0.723674,0.562859,-0.306942,0.588354,0.002418,0.448408,-0.263796,-0.451045,-0.254869,...,-1.318390e-16,4.996004e-16,-1.526557e-16,4.649059e-16,-3.382711e-17,3.764350e-16,-5.134781e-16,8.326673e-16,-0.223390,0
1,-35.548539,-31.850484,-48.325589,15.304184,-113.743307,73.301626,120.589494,-3.872425,-12.005487,6.853897,...,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,102.362243,0
2,-36.802320,-63.344698,-20.645794,16.715537,-20.672064,7.694002,24.956587,-2.687312,-8.423404,1.186360,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,78.235272,0
3,-56.407510,-72.715728,-6.605265,16.491217,34.801666,-26.160506,-19.399981,6.967698,9.537780,3.089395,...,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,4.451791,0
4,-14.904340,12.793880,-7.888739,0.512373,-7.661829,16.614054,-31.764946,-6.290730,-14.741096,-3.038237,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.645814,0


- SMOTE: Oversampling

In [6]:
##### SMOTE
from imblearn.over_sampling import SMOTE

df_features = df.drop(['Class'], axis=1)
df_target = df["Class"]

sm = SMOTE(random_state=12, ratio=1.0)
x_res,y_res = sm.fit_sample(df_features,df_target)

C:\Users\Tiffany Xu\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
df_X_SMOTE = pd.DataFrame(x_res,columns = df_kmeans_notfraud.columns)
df_Y_SMOTE = pd.DataFrame(y_res,columns = ["Class"])
df_SMOTE = pd.concat([df_X_SMOTE,df_Y_SMOTE],axis=1)

In [8]:
df_SMOTE.head()

,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,...,V11_,V12_,V14_,V16_,V17_,V18_,V19_,V21_,normalized_amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.363787,0.090794,-0.551600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,-0.255425,-0.166974,1.612727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,-1.514654,0.207643,0.624501,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.160686,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,-1.387024,-0.054952,-0.226487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.140534,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,0.817739,0.753074,-0.822843,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.073403,0


#### Logistic Regression

In [9]:
X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_Kmeans_under,y_Kmeans_under,test_size = 0.3, random_state = 12)
lr = LogisticRegression()
lr.fit(X_under_train,y_under_train)
y_pred = lr.predict(X_under_test)
tn, fp, fn, tp = confusion_matrix(y_under_test, y_pred).ravel()
print("Kmeans: Logistic Regression Sensitivity/Recall is:",tp/(tp+fn))
print("Kmeans: Logistic Regression Specificity is:",tn/(tn+fp))

Kmeans: Logistic Regression Sensitivity/Recall is: 0.9407894736842105
Kmeans: Logistic Regression Specificity is: 0.9375


In [10]:
x_train_res, x_val_res, y_train_res, y_val_res = train_test_split(x_res,
                                                    y_res,
                                                    test_size = .3,
                                                    random_state=12)

lr = LogisticRegression()
lr.fit(x_train_res,y_train_res)
y_pred = lr.predict(x_val_res)
tn, fp, fn, tp = confusion_matrix(y_val_res, y_pred).ravel()
print("SMOTE: Logistic Regression Sensitivity/Recall is:",tp/(tp+fn))
print("SMOTE: Logistic Regression Specificity is:",tn/(tn+fp))

SMOTE: Logistic Regression Sensitivity/Recall is: 0.9192237421530965
SMOTE: Logistic Regression Specificity is: 0.9706824716859339


In [18]:
#### Kmeans H2OFrame
hf_Kmeans = h2o.H2OFrame(df_Kmeans)
hf_Kmeans["Class"] = hf_Kmeans["Class"].asfactor()

train_Kmeans, valid_Kmeans = hf_Kmeans.split_frame(
    ratios=[0.7], 
    seed=12, 
    destination_frames=['train_Kmeans.hex','valid_Kmeans.hex']
)

#### SMOTE H2OFrame
hf_SMOTE = h2o.H2OFrame(df_SMOTE)
hf_SMOTE["Class"] = hf_SMOTE["Class"].asfactor()

train_SMOTE, valid_SMOTE = hf_SMOTE.split_frame(
    ratios=[0.7], 
    seed=12, 
    destination_frames=['train_SMOTE.hex','valid_SMOTE.hex']
)



Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


#### Gradient Boosting Machine

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [12]:
predictors = hf_Kmeans.columns
del predictors[-1]
response = "Class"

gbm = H2OGradientBoostingEstimator(
  ntrees = 10000,                                                            
  learn_rate = 0.01,                                                         
  stopping_rounds = 20, stopping_tolerance = 1e-4, stopping_metric = "AUC", 
  sample_rate = 0.8,
  col_sample_rate = 0.8,
  seed = 12,
  score_tree_interval = 10)

gbm.train(x=predictors, y=response, training_frame=train_Kmeans,
         validation_frame = valid_Kmeans)

perf = gbm.model_performance(valid = True)
gbm.confusion_matrix(valid = True)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5979462663399188: 


,0,1,Error,Rate
0,144.0,6.0,0.04,(6.0/150.0)
1,2.0,144.0,0.0137,(2.0/146.0)
Total,146.0,150.0,0.027,(8.0/296.0)


In [14]:
print("Kmeans: GBM Sensitivity/Recall is:",144/150)
print("Kmeans: GBM Specificity is:",144/150)

Kmeans: GBM Sensitivity/Recall is: 0.96
Kmeans: GBM Specificity is: 0.96


In [15]:
predictors = hf_SMOTE.columns
del predictors[-1]
response = "Class"

gbm = H2OGradientBoostingEstimator(
  ntrees = 10000,                                               
  learn_rate = 0.01,                                                         
  stopping_rounds = 20, stopping_tolerance = 1e-4, stopping_metric = "AUC", 
  sample_rate = 0.8,
  col_sample_rate = 0.8,
  seed = 12,
  score_tree_interval = 10)

gbm.train(x=predictors, y=response, training_frame=train_SMOTE,
         validation_frame = valid_SMOTE)

perf = gbm.model_performance(valid = True)
gbm.confusion_matrix(valid = True)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4556398035662969: 


,0,1,Error,Rate
0,84629.0,407.0,0.0048,(407.0/85036.0)
1,274.0,85158.0,0.0032,(274.0/85432.0)
Total,84903.0,85565.0,0.004,(681.0/170468.0)


In [16]:
print("SMOTE: GBM Sensitivity/Recall is:",85158/85565)
print("SMOTE: GBM Specificity is:",84629/85036)

SMOTE: GBM Sensitivity/Recall is: 0.9952433822240402
SMOTE: GBM Specificity is: 0.9952137918058234


#### Random Forest Model

In [17]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [23]:
predictors = hf_Kmeans.columns
del predictors[-1]
response = "Class"

rf_kmeans = H2ORandomForestEstimator(
    model_id="rf_kmeans",
    ntrees = 10000, 
    stopping_rounds=20,
    score_each_iteration=True,
    seed=12)

rf_kmeans.train(x=predictors, y=response, training_frame=train_Kmeans,
         validation_frame = valid_Kmeans)

perf = rf_kmeans.model_performance(valid = True)
rf_kmeans.confusion_matrix(valid = True)

drf Model Build progress: |███████████████████████████████████████████████| 100%
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6170212765957447: 


,0,1,Error,Rate
0,146.0,4.0,0.0267,(4.0/150.0)
1,7.0,139.0,0.0479,(7.0/146.0)
Total,153.0,143.0,0.0372,(11.0/296.0)


In [24]:
print("Kmeans: RFM Sensitivity/Recall is:",139/142)
print("Kmeans: RFM Specificity is:",146/150)

Kmeans: RFM Sensitivity/Recall is: 0.9788732394366197
Kmeans: RFM Specificity is: 0.9733333333333334


In [25]:
predictors = hf_Kmeans.columns
del predictors[-1]
response = "Class"

rf_SMOTE = H2ORandomForestEstimator(
    model_id="rf_SMOTE",
    ntrees = 10000,      
    stopping_rounds=20,
    score_each_iteration=True,
    seed=12)

rf_SMOTE.train(x=predictors, y=response, training_frame=train_SMOTE,
         validation_frame = valid_SMOTE)

perf = rf_SMOTE.model_performance(valid = True)
rf_SMOTE.confusion_matrix(valid = True)

drf Model Build progress: |███████████████████████████████████████████████| 100%
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5470289682056039: 


,0,1,Error,Rate
0,85020.0,16.0,0.0002,(16.0/85036.0)
1,11.0,85421.0,0.0001,(11.0/85432.0)
Total,85031.0,85437.0,0.0002,(27.0/170468.0)


In [26]:
print("SMOTE: RFM Sensitivity/Recall is:", 85421/85437)
print("SMOTE: RFM Specificity is:",8502/85036)

SMOTE: RFM Sensitivity/Recall is: 0.9998127275068179
SMOTE: RFM Specificity is: 0.0999811844395315


#### XGBoost

In [47]:
from h2o.estimators import H2OXGBoostEstimator

In [53]:
predictors = hf_Kmeans.columns
del predictors[-1]
response = "Class"

xgb_Kmeans = H2OXGBoostEstimator(
    model_id="xgb_Kmeans",
    ntrees = 10000,
    learn_rate = 0.005,
    sample_rate = 0.1, 
    col_sample_rate = 0.8,
    max_depth = 5,
    nfolds = 3,
    keep_cross_validation_predictions=True,
    stopping_rounds = 10,
    seed = 12,
    distribution = 'bernoulli')

xgb_Kmeans.train(x=predictors, y=response, training_frame=train_Kmeans, validation_frame = valid_Kmeans)

H2OResponseError: Server error water.exceptions.H2ONotFoundArgumentException:
  Error: POST /3/ModelBuilders/xgboost not found
  Request: POST /3/ModelBuilders/xgboost
    data: {'model_id': 'xgb_Kmeans', 'ntrees': '10000', 'learn_rate': '0.005', 'sample_rate': '0.1', 'col_sample_rate': '0.8', 'max_depth': '5', 'nfolds': '3', 'keep_cross_validation_predictions': 'True', 'stopping_rounds': '10', 'seed': '12', 'distribution': 'bernoulli', 'training_frame': 'train_Kmeans.hex', 'validation_frame': 'valid_Kmeans.hex', 'response_column': 'Class'}


In [ ]:
print("Kmeans: XGBoost Sensitivity/Recall is:",139/142)
print("Kmeans: XGBoost Specificity is:",146/150)

In [ ]:
predictors = hf_SMOTE.columns
del predictors[-1]
response = "Class"

xgb_SMOTE = H2OXGBoostEstimator(
    model_id="xgb_SMOTE",
    ntrees = 10000,
    learn_rate = 0.005,
    sample_rate = 0.1, 
    col_sample_rate = 0.8,
    max_depth = 5,
    nfolds = 3,
    keep_cross_validation_predictions=True,
    stopping_rounds = 10,
    seed = 12,
    distribution = 'bernoulli')

xgb_Kmeans.train(x=predictors, y=response, training_frame=train_SMOTE, validation_frame = valid_SMOTE)

In [ ]:
print("SMOTE: XGBoost Sensitivity/Recall is:",139/142)
print("SMOTE: XGBoost Specificity is:",146/150)